# 7506 - Organizacion de Datos - TP N°2

# 1. Librerias

In [2]:
#librerías, no es necesario volverlas a importar
import pandas as pd

import nltk, re, string, collections
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
#Vectorizacion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


%run ../0_Data/Util.ipynb

c:\users\germa\documents\proyectos python\tp1\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\germa\documents\proyectos python\tp1\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\germa\documents\proyectos python\tp1\venv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2. Datos

In [3]:
#Dataset de entranamiento
df_train = pd.read_csv('../dataset/train.csv',index_col='id', encoding='utf8')

#Dataset de test
df_test = pd.read_csv('../dataset/test.csv',index_col='id', encoding='utf8')

In [3]:
df_train.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df_test.head()

,keyword,location,text
id,,,
0,NaN,NaN,Just happened a terrible car crash
2,NaN,NaN,"Heard about #earthquake is different cities, s..."
3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


# 3. Preprocesamiento

In [5]:
def eliminar_puntuacion_excepto_puntos(text):
    punctuationNoPeriod = re.sub("\.","",string.punctuation)
    return text.translate(str.maketrans('','',punctuationNoPeriod))

In [6]:
def preprocesar_df(df, col_name, clean_col_name):
    df[clean_col_name] = df[col_name].apply(lambda x: clean_text_sentiment_analysis(x))

In [7]:
%time preprocesar_df (df_train,'text', 'texto_preprocesado')

Wall time: 3.21 s


In [8]:
df_train.head()

,keyword,location,text,target,texto_preprocesado
id,,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...


In [9]:
%time preprocesar_df (df_test,'text', 'texto_preprocesado')

Wall time: 472 ms


In [10]:
df_test.head()

,keyword,location,text,texto_preprocesado
id,,,,
0,NaN,NaN,Just happened a terrible car crash,happened terrible car crash
2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard earthquake different city stay safe ever...
3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond goose fleeing across str...
9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfire
11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kill china taiwan


In [11]:
df_train_preprocesado = df_train[['texto_preprocesado','target']]
df_train_preprocesado.head()

,texto_preprocesado,target
id,,
1,deed reason earthquake may allah forgive u,1
4,forest fire near la ronge sask canada,1
5,resident asked shelter place notified officer ...,1
6,people receive wildfire evacuation order calif...,1
7,got sent photo ruby alaska smoke wildfire pour...,1


In [12]:
df_test_preprocesado = df_test['texto_preprocesado']

In [13]:
x_train_original = df_train['text']

x_train_preprocesado = df_train['texto_preprocesado']

y_train = df_train['target']

In [14]:
x_test_original = df_test['text']

x_test_preprocesado = df_test['texto_preprocesado']

# 4. Vectorizacion

## 4.1  Obtencion de NGramas, análisis de distintas herramientas

In [15]:
def get_ngrams(documents, size):
    word_vectorizer = CountVectorizer()
    sparse_matrix = word_vectorizer.fit_transform(documents)
    frequencies = sum(sparse_matrix).data
    #frequencies = sum(sparse_matrix).toarray()[0]
    df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
    df = df.sort_values(by='frequency', ascending=False)
    return df

In [16]:
%time get_ngrams(df_train['texto_preprocesado'],1)

Wall time: 2.23 s


,frequency
lga,353
lilourry,347
lich,306
lamha,300
literally,256
...,...
fitness,1
fist,1
fishing,1
fish,1


In [17]:
def documentNgrams(documents, size, count=None):
    ngrams_all = []
    for document in documents:
        tokens = document.split()
        if len(tokens) < size:
            continue
        else:
            output = list(ngrams(tokens, size))
        for ngram in output:
            ngrams_all.append(" ".join(ngram))
    cnt_ngram = Counter()
    for word in ngrams_all:
        cnt_ngram[word] += 1
    df = pd.DataFrame.from_dict(cnt_ngram, orient='index').reset_index()
    df = df.rename(columns={'index':'words', 0:'count'})
    df = df.sort_values(by='count', ascending=False)
    if (count == None):
        return df
    else:
        return(df.head(count))

In [18]:
#elijo este que da mejor tiempo de ejecucion y ademas el otro saltea un par de casos. 
# mas adelante se analiza que los casos que saltea son los de letras solas, por ejemplo "u are" 

%time documentNgrams(df_train['texto_preprocesado'],1, 100)

Wall time: 97.3 ms


,words,count
8,fire,353
126,like,347
56,im,306
305,amp,300
218,get,256
...,...,...
502,even,72
1019,army,72
2080,mass,72
616,thing,72


In [19]:
df_train['tokens'] = df_train['texto_preprocesado'].apply(lambda x : x.split())
df_train.head()

,keyword,location,text,target,texto_preprocesado,tokens
id,,,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,"[deed, reason, earthquake, may, allah, forgive..."
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]"
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...,"[resident, asked, shelter, place, notified, of..."
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...,"[people, receive, wildfire, evacuation, order,..."
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...,"[got, sent, photo, ruby, alaska, smoke, wildfi..."


In [20]:
tokens = df_train.explode('tokens')
tokens.head()

,keyword,location,text,target,texto_preprocesado,tokens
id,,,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,deed
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,reason
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,earthquake
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,may
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,allah


In [21]:
token_list = tokens['tokens'].value_counts().reset_index()

In [22]:
token_list['len'] = token_list['index'].apply(lambda x: len(x))
token_list.head().sort_values('len')

,index,tokens,len
2,im,306,2
3,amp,300,3
4,get,256,3
0,fire,353,4
1,like,347,4


In [23]:
token_list

,index,tokens,len
0,fire,353,4
1,like,347,4
2,im,306,2
3,amp,300,3
4,get,256,3
...,...,...,...
13450,driverless,1,10
13451,amritsar,1,8
13452,biolabs,1,7
13453,palmoil,1,7


In [24]:
vect_list = get_ngrams(df_train['texto_preprocesado'],1).reset_index()['index'].to_list()

In [25]:
token_list.loc[token_list['index'].apply(lambda x : x not in vect_list),:]

,index,tokens,len
13,u,172,1
118,w,65,1
350,x,36,1
418,n,33,1
420,v,33,1
455,b,32,1
570,r,27,1
576,c,26,1
585,k,26,1
760,p,20,1


## 4.2  Obtencion de los 100 BiGramas mas frecuentes para todos los tweets

In [26]:
%time bigrams_all_tweets = documentNgrams(df_train['texto_preprocesado'],2, 100)

Wall time: 107 ms


In [27]:
bigrams_all_tweets.head(10)

,words,count
436,prime minister,108
1559,laughing out,70
1560,out loud,70
37605,suicide bomber,60
7003,burning building,58
5570,body bag,50
965,look like,49
2189,northern california,41
1705,liked video,41
5569,cross body,39


In [28]:
bigrams_list = bigrams_all_tweets.words.to_list()

In [29]:
word_vectorizer_100 = CountVectorizer(ngram_range=(2, 2), vocabulary = bigrams_list)
sparse_matrix = word_vectorizer_100.fit_transform(x_train_preprocesado.head(10))

In [30]:
print (sparse_matrix.toarray()[1][31])

0


In [31]:
bigrams_list[31]

'wild fire'

In [32]:
x_train_preprocesado.iloc[1]

'forest fire near la ronge sask canada'

## 4.3  Obtencion de los 1000 BiGramas mas frecuentes para todos los tweets

In [33]:
%time bigrams_all_tweets_1000 = documentNgrams(df_train['texto_preprocesado'],2, 1000)

Wall time: 111 ms


In [34]:
bigrams_list_1000 = bigrams_all_tweets_1000.words.to_list()

In [35]:
word_vectorizer_1000 = CountVectorizer(ngram_range=(2, 2), vocabulary = bigrams_list_1000)

In [36]:
vectorizer = CountVectorizer(ngram_range = (2,2), max_features = 5) 
vectorizer.fit_transform(x_train_preprocesado)

<7613x5 sparse matrix of type '<class 'numpy.int64'>'
	with 350 stored elements in Compressed Sparse Row format>

In [37]:
vectorizer.get_feature_names()

['burning building',
 'laughing out',
 'out loud',
 'prime minister',
 'suicide bomber']

## 4.4  Obtencion de los BiGramas mas frecuentes para los tweets que son desastres 

In [38]:
%time bigrams_true_tweets = documentNgrams(df_train.loc[(df_train['target']== 1),'texto_preprocesado'],2)

Wall time: 47.9 ms


In [39]:
bigrams_true_tweets

,words,count
230,prime minister,91
15829,suicide bomber,60
875,northern california,41
13720,oil spill,38
1565,california wildfire,36
...,...,...
6955,look grahamwp,1
6956,grahamwp fired,1
6957,fired gun,1
6958,gun arrest,1


## 4.5  Obtencion de los TriGramas mas frecuentes para los tweets que son desastres 

In [40]:
%time trigrams_true_tweets = documentNgrams(df_train.loc[(df_train['target']== 1),'texto_preprocesado'],3)

Wall time: 53 ms


In [41]:
trigrams_true_tweets

,words,count
15141,suicide bomber detonated,30
10229,northern california wildfire,29
13734,latest home razed,28
13735,home razed northern,28
15142,bomber detonated bomb,28
...,...,...
6530,blaze one injured,1
6531,one injured fire,1
6532,crime killeddisplaced million,1
6533,killeddisplaced million systematic,1


## 4.6  Obtencion del vocabulario de los tweets que son desastres naturales  

In [42]:
%time vocabulary_true_tweets = documentNgrams(df_train.loc[(df_train['target']== 1),'texto_preprocesado'],1)

Wall time: 41 ms


In [43]:
vocabulary_true_tweets

,words,count
8,fire,265
327,news,139
355,via,121
45,disaster,118
26,california,115
...,...,...
3775,humphrey,1
3774,hubert,1
3773,dakota,1
3772,economics,1


In [44]:
vocabulary_true_tweets_original = documentNgrams(df_train.loc[(df_train['target']== 1),'text'],1)

In [45]:
vocabulary_true_tweets_original

,words,count
3,the,1051
25,in,1037
5,of,888
55,a,730
23,to,706
...,...,...
7028,https://t.co/X17fUBQBGG,1
7029,@_jeesss_,1
7030,@Ethereal_7,1
7033,medic,1
